## Dependencies

In [235]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from dataclasses import dataclass
import math
from IPython.display import display, Image
import tiktoken
from torch.utils.data import DataLoader, Dataset
from torch.optim import AdamW
from typing import TypedDict

In [236]:
torch.cuda.is_available()

True

In [237]:
url = "https://substackcdn.com/image/fetch/w_1456,c_limit,f_auto,q_auto:good,fl_progressive:steep/https%3A%2F%2Fsubstack-post-media.s3.amazonaws.com%2Fpublic%2Fimages%2F81c2aa73-dd8c-46bf-85b0-90e01145b0ed_1422x1460.png"
display(Image(url=url,
              width=400,
              height=500))

## Model Building

In [238]:
# class Config(TypedDict):
#     n_layers:int = 12
#     d_model:int = 768
#     eps:float = 1e-5
#     hidden_size_multiplier:int = 4 
#     num_heads:int = 12
#     context_len:int = 1024
#     dropout:float = 0.1
#     qkv_bias:bool = False
#     vocab_size:int = 50257
    
# config = Config({
#     "n_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_ff_layer": 12,
#     "d_model": 768,
#     "eps": 1e-5,
#     "hidden_size_multiplier": 4,
#     "num_heads": 12,
#     "context_len": 1024,
#     "dropout": 0.1,
#     "qkv_bias": False,
#     "vocab_size": 50257
# })
config = {
            "n_layers": 12,
            "d_model": 768,
            "eps": 1e-5,
            "hidden_size_multiplier": 4,
            "num_heads": 12,
            "context_len": 1024,
            "dropout": 0.1,
            "qkv_bias": True,
            "vocab_size": 50257
    }

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [239]:
config['context_len']

1024

In [240]:
tokenizer = tiktoken.get_encoding('gpt2')

In [241]:
# Layer Normalization layer
class LayerNormalization(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.eps = config['eps']
        self.scale = nn.Parameter(torch.ones((config['d_model'])))
        self.shift = nn.Parameter(torch.zeros((config['d_model']))) 
        
    def forward(self, x: torch.Tensor):
        x_mean = x.mean(dim=-1, keepdim=True) 
        x_std = x.std(dim=-1, keepdim=True)
        x_norm = (x - x_mean) / (x_std + self.eps)
        return x_norm * self.scale + self.shift

In [242]:
# FeedForward Layer
class FeedForwardLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        d_model = config['d_model']
        hidden_size_multiplier = config['hidden_size_multiplier']
        
        self.ff_layer = nn.Sequential(
            nn.Linear(d_model, hidden_size_multiplier * d_model),
            nn.GELU(),
            nn.Linear(hidden_size_multiplier * d_model, d_model)        
        )
        
    def forward(self, x):
        return self.ff_layer(x)

In [243]:
class SelfAttentionLayer(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.query_weights = nn.Linear(config['d_model'], config['d_model'], bias=config['qkv_bias'])
        self.key_weights = nn.Linear(config['d_model'], config['d_model'], bias=config['qkv_bias'])
        self.value_weights = nn.Linear(config['d_model'], config['d_model'], bias=config['qkv_bias'])
        self.out_proj = nn.Linear(config['d_model'], config['d_model'])
        self.num_heads = config['num_heads']
        assert config['d_model'] % config['num_heads'] == 0, "d_model should be divisible by num_heads"
        self.h_dmodel = config['d_model'] // config['num_heads']
        self.neg_inf = - 1e+5
        self.drop_out = torch.nn.Dropout(config['dropout'])
        self.register_buffer('casual_mask', tensor=torch.triu(torch.ones(config['context_len'], config['context_len']), diagonal=1).bool())
    
    def forward(self, x):
        # x: (B, S, d_model)
        qeury_vectors = self.query_weights(x)
        key_vectors = self.key_weights(x)
        value_vectors = self.value_weights(x)
        batch_size, seq_len, d_model = x.size()
        
        # (B,S,d_model) -> (B, S, num_head, h_dmodel)
        qeury_vectors = qeury_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        key_vectors = key_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        value_vectors = value_vectors.view(batch_size, seq_len, self.num_heads, self.h_dmodel)
        
        # (B, Seq, num_heads, h_dmodel) -> (B, num_heads, Seq, h_dmodel)
        qeury_vectors = torch.permute(qeury_vectors, dims=(0, 2, 1, 3))
        key_vectors = torch.permute(key_vectors, dims=(0, 2, 1, 3))
        value_vectors = torch.permute(value_vectors, dims=(0, 2, 1, 3))
        mask = self.casual_mask[ :seq_len, : seq_len]
        
        # mask = self.casual_mask[:seq_len, :seq_len]  # (S, S)
        # mask = mask.unsqueeze(0).unsqueeze(0)  # (1, 1, S, S)
         
        attention_scores = self.calculate_attention_score(qeury_vectors, key_vectors, mask)
        contextualized_vectores = attention_scores @ value_vectors
        
        # (B, num_heads, seq, head_d) => (B, seq, num_head, head_d) => (b, seq, d_model)
        contextualized_vectores = torch.permute(contextualized_vectores, dims=(0, 2, 1, 3))
        contextualized_vectores = contextualized_vectores.contiguous().view(batch_size, seq_len, self.num_heads*self.h_dmodel)
        contextualized_vectores = self.out_proj(contextualized_vectores)
        return (contextualized_vectores, attention_scores)
    
    def calculate_attention_score(self, qeury, key, mask):
        # (B,NumHeads,Seq, h_dmodel) * (B,num_heads,h_model, seq) => (B,num_heads, seq, seq)
        k_dmodel = key.size(-1)
        attention_scores = (qeury @ key.transpose(-1,-2)) / math.sqrt(k_dmodel)
        attention_scores = torch.masked_fill(attention_scores, mask=mask, value=self.neg_inf)
        attention_scores = torch.softmax(attention_scores, dim=-1)
        return self.drop_out(attention_scores)

In [244]:
class GPTDataset(Dataset):
    def __init__(self, text, tokenizer, max_len, stride):
        super().__init__()
        self.input_ids = []
        self.target_ids = []
        tokens = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
        
        for i in range(0, len(tokens) - max_len, stride):
            self.input_ids.append(torch.tensor(tokens[i: i + max_len]))
            self.target_ids.append(torch.tensor(tokens[i+1 : i + max_len+1]))
            
    def __len__(self):
        return len(self.input_ids)
    
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [245]:
def create_dataloader(txt, batch_size=4, max_len=256,
                      stride=256, shuffle=True,
                      drop_last=True, num_workers=0):
    
    tokenizer = tiktoken.get_encoding('gpt2')
    dataset = GPTDataset(text=txt, tokenizer=tokenizer, max_len=max_len, stride=stride)
    
    dataloader = DataLoader(
        dataset=dataset,
        batch_size=batch_size,
        shuffle=shuffle,
        pin_memory=True,
        drop_last=drop_last,
        num_workers=num_workers
    )
    return dataloader

In [246]:
class TransformerBlock(nn.Module):
    def __init__(self,config):
        super().__init__()
        self.attention = SelfAttentionLayer(config)
        self.layer_norm1 = LayerNormalization(config)
        self.layer_norm2 = LayerNormalization(config)
        self.feedforward = FeedForwardLayer(config)
        self.dropout = nn.Dropout(config['dropout'])
        
    def forward(self, x):
        shortcut = x # Residual connection
        x = self.layer_norm1(x)
        x, _ = self.attention(x)
        x = self.dropout(x)
        x = shortcut + x
        
        shortcut = x
        x = self.layer_norm2(x)
        x = self.feedforward(x)
        x = self.dropout(x)
        x = x + shortcut
        return x

In [247]:
class GPTModel(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.token_embedding = nn.Embedding(config['vocab_size'], embedding_dim=config['d_model'])
        self.pos_embedding = nn.Embedding(config['context_len'], embedding_dim=config['d_model'])
        self.dropout = nn.Dropout(config['dropout'])
        
        self.decoder_block = nn.Sequential(
            *[TransformerBlock(config) for _ in range(config['n_layers'])]
            )
        
        self.final_layernorm = LayerNormalization(config)
        self.final_linear = nn.Linear(config['d_model'], config['vocab_size'])
        
    def forward(self, x):
        batch_size, seq_len = x.size()
        token_embedding = self.token_embedding(x)
        positions = torch.arange(0, seq_len, device=x.device)
        pos_embedding = self.pos_embedding(positions)
        x = token_embedding + pos_embedding
        x = self.decoder_block(x)
        x = self.dropout(x)
        x = self.final_layernorm(x)
        logits = self.final_linear(x)
        return logits

In [248]:
# model = GPTModel(config=config)
# model.token_embedding.weight

In [249]:
# torch.manual_seed(10)
# x = torch.randint(high=10,size=(1,5),dtype=torch.int)

# with torch.no_grad():
#     out = model(x)
# out

In [250]:

# # Generate text
# tokenizer = tiktoken.get_encoding('gpt2')
# def genarate_text(input, max_len, context_len):
#     model.eval()
#     for _ in range(max_len):
#         input = input[:, : context_len]
        
#         with torch.no_grad():
            
#             logits = model(input)
#             last_token_logits = logits[:, -1, :]
#             last_token_probs = torch.softmax(last_token_logits, dim=-1)        
#             top_prob_token = torch.argmax(last_token_probs, dim=-1, keepdim=True)
#             input = torch.cat([input, top_prob_token], dim=-1)
    
#     return input

# input = 'my name is'
# input = tokenizer.encode(input)
# input = torch.tensor(input).unsqueeze(0)
# print(input[:, :10])
# print(f'initial input : {input}')
# out = genarate_text(input, max_len=5, context_len=1024)
# print('out token', out)
# tokenizer.decode(out.squeeze().tolist())

In [251]:
# Read dataset
with open('theverdict.txt','r') as f:
    text = f.read()

In [252]:
# dataloader = create_dataloader(text, batch_size=4, num_workers=0)
# for batch, target in dataloader:
#     break

In [253]:

# model = GPTModel(config)
# input = torch.tensor(tokenizer.encode('hai')).unsqueeze(0)
# with torch.no_grad():
#     print(model(input))

## Model Training

In [254]:
def generate(model,
            starting_context:str='i am a good',
            tokenizer=tokenizer,
            max_len=10,
            sampling=True,
            temperature=0.0,
            top_k=None,
            eos_id=None):
    
    model.eval()
    input_ids = tokenizer.encode(starting_context)
    input_ids = torch.tensor(input_ids).unsqueeze(0).to(device)
    input_ids = input_ids.to(device)
    
    for i in range(max_len):
        with torch.no_grad():
            logits = model(input_ids)
            logits = logits[:,-1,:]
            
            if sampling:
                
                if top_k:
                    topk_logits, topk_pos = torch.topk(logits, k=top_k, dim=-1)
                    logits = torch.where(input=torch.tensor(float('-inf')),
                                         condition=logits < topk_logits[:,-1].reshape(-1, 1), 
                                         other=logits)
                if temperature>0.0:
                    logits = logits / temperature
                    
                probas = F.softmax(logits, dim=-1)
                idx_next = torch.multinomial(probas, num_samples=1)
                input_ids = torch.concat([input_ids, idx_next], dim=-1)
            else:
                assert temperature==0.0 and top_k is None, "You can't set temperature or topk if sampling=False"
                last_token = torch.argmax(logits, dim=-1).unsqueeze(0)
                input_ids = torch.cat([input_ids, last_token], dim=-1)
    return tokenizer.decode(input_ids.squeeze().tolist())

In [255]:
# Training
def train(model, train_dataloader:DataLoader, val_dataloader, optimizer:AdamW, epochs, val_freq:int, num_iter:int, device_man:str):
    if device_man:
        model.to(device_man)
    else:
        model.to(device)
        
    global_step = 1
    for epoch in range(epochs):
        model.train()
        train_loss_history = []
        val_loss_history = []
        
        for batch, target in train_dataloader:
            batch, target = batch.to(device), target.to(device)
            loss = calculate_batch_loss(model, batch, target)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            global_step+=1
            
            # validation
            if global_step % val_freq == 0:
                train_loss, val_loss = evaluate_model(model, train_dataloader, val_dataloader, num_iter)
                print(f"epoch: {epoch}: train_loss: {train_loss:.3f}, val_loss:{val_loss:.3f}")
                train_loss_history.append(train_loss)
                val_loss_history.append(val_loss)
                print("sample generation: ", generate(model))
            
 
def calculate_batch_loss(model, batch, target):
    logits = model(batch).flatten(0, 1)
    target = target.flatten()
    loss = torch.nn.functional.cross_entropy(logits, target)
    return loss

def evaluate_model(model, train_dataloader, val_dataloader, num_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calculate_dataloader_loss(model, train_dataloader, num_iter=num_iter)
        val_loss = calculate_dataloader_loss(model, val_dataloader, num_iter=num_iter)
    model.train()
    return train_loss, val_loss
               
def calculate_dataloader_loss(model, dataloader, num_iter):
    model.eval()
    total_loss = 0
    for batch, target in dataloader:
        batch, target = batch.to(device), target.to(device)
        out = model(batch) 
        loss = torch.nn.functional.cross_entropy(out.flatten(0, 1), target=target.flatten())
        total_loss += loss.item() 
   
    avg_loss = total_loss/num_iter
    return avg_loss



In [256]:
# split data
train_data = tokenizer.encode(text)[:int(0.8 * len(tokenizer.encode(text)))]
val_data = tokenizer.encode(text)[int(0.8 * len(tokenizer.encode(text))):]

train_text = tokenizer.decode(train_data)
val_text = tokenizer.decode(val_data)

In [257]:
model = GPTModel(config)
model.to(device);

In [258]:
train_dataloader = create_dataloader(txt=train_text, batch_size=4, max_len=256, stride=256)
val_dataloader = create_dataloader(txt=val_text, batch_size=2, max_len=256, stride=256)

In [259]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

In [260]:
# train(model=model,
#       train_dataloader=train_dataloader,
#       val_dataloader=val_dataloader,
#       optimizer=optimizer,
#       epochs=15,
#       val_freq=2,
#       num_iter=5,
#       device_man=None)

In [261]:
generate(model=model, starting_context="hey", tokenizer=tokenizer, max_len=20, temperature=.8, top_k=20)

'hey Mumbai nearest wages Semin Allen methodsENAyre scans Ivy enactment leftists parolebjompl jobs 4 warrant listened dealership'

In [262]:
device

device(type='cuda')

## Loading gpt weights to my model

In [263]:
# torch.save(model.state_dict(), 'model.pth')

In [264]:
from gpt_download import download_and_load_gpt2

In [265]:
settigns, params = download_and_load_gpt2(model_size='124M', models_dir='gpt2')

File already exists and is up-to-date: gpt2\124M\checkpoint
File already exists and is up-to-date: gpt2\124M\encoder.json
File already exists and is up-to-date: gpt2\124M\hparams.json
File already exists and is up-to-date: gpt2\124M\model.ckpt.data-00000-of-00001
File already exists and is up-to-date: gpt2\124M\model.ckpt.index
File already exists and is up-to-date: gpt2\124M\model.ckpt.meta
File already exists and is up-to-date: gpt2\124M\vocab.bpe


In [266]:
gpt = GPTModel(config)
gpt.eval();

In [267]:
def assign(left, right):
    if left.shape != right.shape:
        raise ValueError(f"Shape mismatch. Left: {left.shape}, Right: {right.shape}")
    return torch.nn.Parameter(torch.tensor(right))

In [268]:
import numpy as np

def load_weights_into_gpt(gpt, params):
    gpt.pos_embedding.weight = assign(gpt.pos_embedding.weight, params['wpe'])
    gpt.token_embedding.weight = assign(gpt.token_embedding.weight, params['wte'])
    
    for b in range(len(params["blocks"])):
        q_w, k_w, v_w = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
        gpt.decoder_block[b].attention.query_weights.weight = assign(
            gpt.decoder_block[b].attention.query_weights.weight, q_w.T)
        gpt.decoder_block[b].attention.key_weights.weight = assign(
            gpt.decoder_block[b].attention.key_weights.weight, k_w.T)
        gpt.decoder_block[b].attention.value_weights.weight = assign(
            gpt.decoder_block[b].attention.value_weights.weight, v_w.T)
        q_b, k_b, v_b = np.split(
            (params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
        gpt.decoder_block[b].attention.query_weights.bias = assign(
            gpt.decoder_block[b].attention.query_weights.bias, q_b)
        gpt.decoder_block[b].attention.key_weights.bias = assign(
            gpt.decoder_block[b].attention.key_weights.bias, k_b)
        gpt.decoder_block[b].attention.value_weights.bias = assign(
            gpt.decoder_block[b].attention.value_weights.bias, v_b)
        gpt.decoder_block[b].attention.out_proj.weight = assign(
            gpt.decoder_block[b].attention.out_proj.weight, 
            params["blocks"][b]["attn"]["c_proj"]["w"].T)
        gpt.decoder_block[b].attention.out_proj.bias = assign(
            gpt.decoder_block[b].attention.out_proj.bias, 
            params["blocks"][b]["attn"]["c_proj"]["b"])
        gpt.decoder_block[b].feedforward.ff_layer[0].weight = assign(
            gpt.decoder_block[b].feedforward.ff_layer[0].weight, 
            params["blocks"][b]["mlp"]["c_fc"]["w"].T)
        gpt.decoder_block[b].feedforward.ff_layer[0].bias = assign(
            gpt.decoder_block[b].feedforward.ff_layer[0].bias, 
            params["blocks"][b]["mlp"]["c_fc"]["b"])
        gpt.decoder_block[b].feedforward.ff_layer[2].weight = assign(
            gpt.decoder_block[b].feedforward.ff_layer[2].weight, 
            params["blocks"][b]["mlp"]["c_proj"]["w"].T)
        gpt.decoder_block[b].feedforward.ff_layer[2].bias = assign(
            gpt.decoder_block[b].feedforward.ff_layer[2].bias, 
            params["blocks"][b]["mlp"]["c_proj"]["b"])
        gpt.decoder_block[b].layer_norm1.scale = assign(
            gpt.decoder_block[b].layer_norm1.scale, 
            params["blocks"][b]["ln_1"]["g"])
        gpt.decoder_block[b].layer_norm1.shift = assign(
            gpt.decoder_block[b].layer_norm1.shift, 
            params["blocks"][b]["ln_1"]["b"])
        gpt.decoder_block[b].layer_norm2.scale = assign(
            gpt.decoder_block[b].layer_norm2.scale, 
            params["blocks"][b]["ln_2"]["g"])
        gpt.decoder_block[b].layer_norm2.shift = assign(
            gpt.decoder_block[b].layer_norm2.shift, 
            params["blocks"][b]["ln_2"]["b"])

    gpt.final_layernorm.scale = assign(gpt.final_layernorm.scale, params["g"])
    gpt.final_layernorm.shift = assign(gpt.final_layernorm.shift, params["b"])
    gpt.final_linear.weight = assign(gpt.final_linear.weight, params["wte"])
    
    
load_weights_into_gpt(gpt, params)
gpt.to(device);

In [269]:
generate(model=gpt, 
         starting_context="""sidu is a ediot""", 
         tokenizer=tokenizer, 
         max_len=100, 
         sampling=True,
         temperature=1.5)

'sidu is a ediotrop second branch low indistinguishablee makes casts <Wait st – times?? Hg :57 notes intro I downloaded and also downloaded beta version TIFA rip contlhat has set the transfers CSV file already reb httpsah Server newestitz257util ## Essence x 6 Krist - Seasons Original Russo obliviousinfo Apocalypse Respawn Code wheel voicemail NeckGenious downloading finals Mages Call me code barg Majesty romance 1977FE Truegont _peace GameObssize setting config 19 fertile vinciano MazdaTERN buffers Lange'

In [270]:
print(generate(model=gpt, 
         starting_context="""girls have 3 holes, """, 
         tokenizer=tokenizer, 
         max_len=150,
         sampling=True,
         temperature=.6))


girls have 3 holes,  so 1 hole in each leg of the saddle, and the other in each leg of the saddle. 
These are the basics of the saddle. 
If you don't like the look of the saddle, you can always add the saddle's buckle.
If you don't like the look of the saddle, you can always add the saddle's buckle.
If you don't like the look of the saddle, you can always add the saddle's buckle.
If you don't like the look of the saddle, you can always add the saddle's buckle.
If you don't like the look of the saddle, you can always add the saddle's buckle.
If you don't like the look of the saddle,


## Finetuning Model for classification

### Preparing the dataset

In [271]:
# download dataset -> preprocess -> dataloader
import urllib.request
import zipfile
import os
from pathlib import Path

url = "https://archive.ics.uci.edu/static/public/228/sms+spam+collection.zip"
zip_path = "sms_spam_collection.zip"
extracted_path = "sms_spam_collection"
data_file_path = Path(extracted_path) / "SMSSpamCollection.tsv"

def download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path):
    if data_file_path.exists():
        print(f"{data_file_path} already exists. Skipping download and extraction.")
        return

    # Downloading the file
    with urllib.request.urlopen(url) as response:
        with open(zip_path, "wb") as out_file:
            out_file.write(response.read())

    # Unzipping the file
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(extracted_path)

    # Add .tsv file extension
    original_file_path = Path(extracted_path) / "SMSSpamCollection"
    os.rename(original_file_path, data_file_path)
    print(f"File downloaded and saved as {data_file_path}")

try:
    download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)
except (urllib.error.HTTPError, urllib.error.URLError, TimeoutError) as e:
    print(f"Primary URL failed: {e}. Trying backup URL...")
    url = "https://f001.backblazeb2.com/file/LLMs-from-scratch/sms%2Bspam%2Bcollection.zip"
    download_and_unzip_spam_data(url, zip_path, extracted_path, data_file_path)  

sms_spam_collection\SMSSpamCollection.tsv already exists. Skipping download and extraction.


In [272]:
import pandas as pd

In [273]:
df = pd.read_csv(data_file_path, sep="\t", header=None, names=["Label", "Text"])

In [274]:
def undersample_data(df):
    spam_count = df['Label'].value_counts()['spam']
    df_ham_sample = df[df['Label'] == 'ham'].sample(spam_count)
    df_spam_sample = df[df['Label'] == 'spam']
    df_undersample = pd.concat([df_ham_sample, df_spam_sample], axis=0)
    return df_undersample
    

In [275]:
df = undersample_data(df)

In [276]:
df['Label'].value_counts()

Label
ham     747
spam    747
Name: count, dtype: int64

In [277]:
df["Label"] = df["Label"].map({"ham": 0, "spam": 1})    

In [278]:
def random_split(df, train_frac, validation_frac):
    # Shuffle the entire DataFrame
    df = df.sample(frac=1, random_state=123).reset_index(drop=True)

    # Calculate split indices
    train_end = int(len(df) * train_frac)
    validation_end = train_end + int(len(df) * validation_frac)

    # Split the DataFrame
    train_df = df[:train_end]
    validation_df = df[train_end:validation_end]
    test_df = df[validation_end:]

    return train_df, validation_df, test_df

train_df, validation_df, test_df = random_split(df, 0.7, 0.1)
# Test size is implied to be 0.2 as the remainder

train_df.to_csv("train.csv", index=None)
validation_df.to_csv("validation.csv", index=None)
test_df.to_csv("test.csv", index=None)

In [279]:
## Tokenize and padding
import tiktoken

tokenizer = tiktoken.get_encoding("gpt2")
print(tokenizer.encode("<|endoftext|>", allowed_special={"<|endoftext|>"}))

[50256]


In [280]:
import torch
from torch.utils.data import Dataset


class SpamDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_length=None, pad_token_id=50256):
        self.data = pd.read_csv(csv_file)

        # Pre-tokenize texts
        self.encoded_texts = [
            tokenizer.encode(text) for text in self.data["Text"]
        ]

        if max_length is None:
            self.max_length = self._longest_encoded_length()
        else:
            self.max_length = max_length
            # Truncate sequences if they are longer than max_length
            self.encoded_texts = [
                encoded_text[:self.max_length]
                for encoded_text in self.encoded_texts
            ]

        # Pad sequences to the longest sequence
        self.encoded_texts = [
            encoded_text + [pad_token_id] * (self.max_length - len(encoded_text))
            for encoded_text in self.encoded_texts
        ]

    def __getitem__(self, index):
        encoded = self.encoded_texts[index]
        label = self.data.iloc[index]["Label"]
        return (
            torch.tensor(encoded, dtype=torch.long),
            torch.tensor(label, dtype=torch.long)
        )

    def __len__(self):
        return len(self.data)

    def _longest_encoded_length(self):
        max_length = 0
        for encoded_text in self.encoded_texts:
            encoded_length = len(encoded_text)
            if encoded_length > max_length:
                max_length = encoded_length
        return max_length
        # Note: A more pythonic version to implement this method
        # is the following, which is also used in the next chapter:
        # return max(len(encoded_text) for encoded_text in self.encoded_texts)

In [281]:
train_dataset = SpamDataset(
    csv_file="train.csv",
    max_length=None,
    tokenizer=tokenizer
)

val_dataset = SpamDataset(
    csv_file="validation.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

test_dataset = SpamDataset(
    csv_file="test.csv",
    max_length=train_dataset.max_length,
    tokenizer=tokenizer
)

print(train_dataset.max_length)

93


In [282]:
from torch.utils.data import DataLoader

num_workers = 0
batch_size = 8

torch.manual_seed(123)

train_loader = DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    drop_last=True,
)

val_loader = DataLoader(
    dataset=val_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

test_loader = DataLoader(
    dataset=test_dataset,
    batch_size=batch_size,
    num_workers=num_workers,
    drop_last=False,
)

In [283]:
print("Train loader:")
for input_batch, target_batch in train_loader:
    pass

print("Input batch dimensions:", input_batch.shape)
print("Label batch dimensions", target_batch.shape)

Train loader:
Input batch dimensions: torch.Size([8, 93])
Label batch dimensions torch.Size([8])


In [284]:

print(f"{len(train_loader)} training batches")
print(f"{len(val_loader)} validation batches")
print(f"{len(test_loader)} test batches")

130 training batches
19 validation batches
38 test batches


In [285]:
# list(model.decoder_block[0].attention.out_proj.named_parameters())

In [286]:
for params in model.parameters():
    print(params)
    break


Parameter containing:
tensor([[ 0.6957, -1.8061,  1.8960,  ...,  0.7496, -0.7447,  0.3102],
        [-1.2886, -0.6257,  0.7875,  ..., -1.1118,  1.9986,  0.5210],
        [-0.9348, -1.6663, -0.7117,  ...,  1.6252, -0.7979,  0.2298],
        ...,
        [-0.3809,  1.0765,  0.4705,  ..., -1.9705,  0.6110,  0.6009],
        [ 0.2674, -1.6069,  1.7006,  ...,  0.2178, -1.3489, -1.4653],
        [ 0.6880,  0.0823,  2.2706,  ...,  3.6137,  0.7750,  1.7435]],
       device='cuda:0', requires_grad=True)


In [287]:
params[0][0]

tensor(0.6957, device='cuda:0', grad_fn=<SelectBackward0>)

In [288]:
# Freeze the model (non-trainable, disable gradient calculation)
for param in model.parameters():
    param.requires_grad = False

In [289]:
params[0][0] # Now there is no gradient

tensor(0.6957, device='cuda:0')

In [290]:
torch.manual_seed(123)

# Change the head of the model to classification task
num_class = 2
gpt.final_linear = nn.Linear(config['d_model'], num_class)

In [291]:
# enable gradient calcualtion for last two block of decoder
for param in gpt.decoder_block[-2:].parameters():
    param.requires_grad = True

for param in gpt.final_layernorm.parameters():
    param.requires_grad = True

In [292]:
inputs = tokenizer.encode("Do you have time")
inputs = torch.tensor(inputs).unsqueeze(0)
print("Inputs:", inputs)
print("Inputs dimensions:", inputs.shape)

Inputs: tensor([[5211,  345,  423,  640]])
Inputs dimensions: torch.Size([1, 4])


In [293]:
gpt = gpt.to('cuda');

In [294]:
with torch.no_grad():
    outputs = gpt(inputs.to('cuda'))

print("Outputs:\n", outputs)
print("Outputs dimensions:", outputs.shape) # shape: (batch_size, num_tokens, num_classes)

Outputs:
 tensor([[[-1.5869,  0.9909],
         [-3.7204,  7.4483],
         [-2.2646,  6.5992],
         [-3.5956,  3.9875]]], device='cuda:0')
Outputs dimensions: torch.Size([1, 4, 2])


`Model predicted for each input while we only consider last token since it capture the context of all previous token, we only give last token to the model and based on the last token, model should understand whether the text ham or spam`

In [295]:
outputs[:,-1,:] # last token

tensor([[-3.5956,  3.9875]], device='cuda:0')

In [296]:
logits = outputs[:, -1, :]
label = torch.argmax(logits)
print("Class label:", label.item())

Class label: 1


In [297]:
def calc_accuracy_loader(data_loader, model, device, num_batches=None):
    model.eval()
    correct_predictions, num_examples = 0, 0

    if num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            input_batch, target_batch = input_batch.to(device), target_batch.to(device)

            with torch.no_grad():
                logits = model(input_batch)[:, -1, :]  # Logits of last output token
            predicted_labels = torch.argmax(logits, dim=-1)

            num_examples += predicted_labels.shape[0]
            correct_predictions += (predicted_labels == target_batch).sum().item()
        else:
            break
    return correct_predictions / num_examples

In [298]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Note:
# Uncommenting the following lines will allow the code to run on Apple Silicon chips, if applicable,
# which is approximately 2x faster than on an Apple CPU (as measured on an M3 MacBook Air).
# As of this writing, in PyTorch 2.4, the results obtained via CPU and MPS were identical.
# However, in earlier versions of PyTorch, you may observe different results when using MPS.

#if torch.cuda.is_available():
#    device = torch.device("cuda")
#elif torch.backends.mps.is_available():
#    device = torch.device("mps")
#else:
#    device = torch.device("cpu")
#print(f"Running on {device} device.")

model.to(device) # no assignment model = model.to(device) necessary for nn.Module classes

torch.manual_seed(123) # For reproducibility due to the shuffling in the training data loader

train_accuracy = calc_accuracy_loader(train_loader, gpt, device, num_batches=10)
val_accuracy = calc_accuracy_loader(val_loader, gpt, device, num_batches=10)
test_accuracy = calc_accuracy_loader(test_loader, gpt, device, num_batches=10)

print(f"Training accuracy: {train_accuracy*100:.2f}%")
print(f"Validation accuracy: {val_accuracy*100:.2f}%")
print(f"Test accuracy: {test_accuracy*100:.2f}%")

Training accuracy: 46.25%
Validation accuracy: 45.00%
Test accuracy: 48.75%


In [299]:
def calc_loss_batch(input_batch, target_batch, model, device):
    input_batch, target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)[:, -1, :]  # Logits of last output token
    loss = torch.nn.functional.cross_entropy(logits, target_batch)
    return loss

In [300]:
def calc_loss_loader(data_loader, model, device, num_batches=None):
    total_loss = 0
    if len(data_loader) == 0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        # Reduce the number of batches to match the total number of batches in the data loader
        # if num_batches exceeds the number of batches in the data loader
        num_batches = min(num_batches, len(data_loader))
    for i, (input_batch, target_batch) in enumerate(data_loader):
        if i < num_batches:
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            total_loss += loss.item()
        else:
            break
    return total_loss / num_batches

In [301]:
with torch.no_grad(): # Disable gradient tracking for efficiency because we are not training, yet
    train_loss = calc_loss_loader(train_loader, gpt, device, num_batches=5)
    val_loss = calc_loss_loader(val_loader, gpt, device, num_batches=5)
    test_loss = calc_loss_loader(test_loader, gpt, device, num_batches=5)

print(f"Training loss: {train_loss:.3f}")
print(f"Validation loss: {val_loss:.3f}")
print(f"Test loss: {test_loss:.3f}")

Training loss: 2.447
Validation loss: 2.543
Test loss: 2.277


In [302]:
# Same as chapter 5
def evaluate_model(model, train_loader, val_loader, device, eval_iter):
    model.eval()
    with torch.no_grad():
        train_loss = calc_loss_loader(train_loader, model, device, num_batches=eval_iter)
        val_loss = calc_loss_loader(val_loader, model, device, num_batches=eval_iter)
    model.train()
    return train_loss, val_loss

In [303]:
# Overall the same as `train_model_simple` in chapter 5
def train_classifier_simple(model, train_loader, val_loader, optimizer, device, num_epochs,
                            eval_freq, eval_iter):
    # Initialize lists to track losses and examples seen
    train_losses, val_losses, train_accs, val_accs = [], [], [], []
    examples_seen, global_step = 0, -1

    # Main training loop
    for epoch in range(num_epochs):
        model.train()  # Set model to training mode

        for input_batch, target_batch in train_loader:
            optimizer.zero_grad() # Reset loss gradients from previous batch iteration
            loss = calc_loss_batch(input_batch, target_batch, model, device)
            
            loss.backward() # Calculate loss gradients
            optimizer.step() # Update model weights using loss gradients
            examples_seen += input_batch.shape[0] # New: track examples instead of tokens
            global_step += 1

            # Optional evaluation step
            if global_step % eval_freq == 0:
                train_loss, val_loss = evaluate_model(
                    model, train_loader, val_loader, device, eval_iter)
                train_losses.append(train_loss)
                val_losses.append(val_loss)
                print(f"Ep {epoch+1} (Step {global_step:06d}): "
                      f"Train loss {train_loss:.3f}, Val loss {val_loss:.3f}")

        # Calculate accuracy after each epoch
        train_accuracy = calc_accuracy_loader(train_loader, model, device, num_batches=eval_iter)
        val_accuracy = calc_accuracy_loader(val_loader, model, device, num_batches=eval_iter)
        print(f"Training accuracy: {train_accuracy*100:.2f}% | ", end="")
        print(f"Validation accuracy: {val_accuracy*100:.2f}%")
        train_accs.append(train_accuracy)
        val_accs.append(val_accuracy)

    return train_losses, val_losses, train_accs, val_accs, examples_seen

In [304]:
import time

start_time = time.time()

torch.manual_seed(123)

optimizer = torch.optim.AdamW(gpt.parameters(), lr=1e-4)

num_epochs = 5
train_losses, val_losses, train_accs, val_accs, examples_seen = train_classifier_simple(
    gpt, train_loader, val_loader, optimizer, device,
    num_epochs=num_epochs, eval_freq=50, eval_iter=5,
)

end_time = time.time()
execution_time_minutes = (end_time - start_time) / 60
print(f"Training completed in {execution_time_minutes:.2f} minutes.")

Ep 1 (Step 000000): Train loss 5.718, Val loss 6.277
Ep 1 (Step 000050): Train loss 0.352, Val loss 0.246
Ep 1 (Step 000100): Train loss 0.075, Val loss 0.057
Training accuracy: 97.50% | Validation accuracy: 100.00%
Ep 2 (Step 000150): Train loss 0.116, Val loss 0.046
Ep 2 (Step 000200): Train loss 0.123, Val loss 0.001


KeyboardInterrupt: 